In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/feedback-prize-english-language-learning/sample_submission.csv
/kaggle/input/feedback-prize-english-language-learning/train.csv
/kaggle/input/feedback-prize-english-language-learning/test.csv


In [4]:
import pandas as pd
import numpy as np

In [34]:
import tensorflow as tf

In [20]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
from sklearn.metrics import mean_squared_error

In [25]:
from tqdm import tqdm

In [6]:
train = pd.read_csv("../input/feedback-prize-english-language-learning/train.csv")
test = pd.read_csv("../input/feedback-prize-english-language-learning/test.csv")
sample_submission = pd.read_csv("../input/feedback-prize-english-language-learning/sample_submission.csv")

In [35]:
def mcrmse(y_true, y_pred):
    colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=0)
    return tf.reduce_mean(tf.sqrt(colwise_mse), axis=0)

In [7]:
train.head(5)

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


In [8]:
test

,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


In [10]:
FEATURE_COLUMNS = ['full_text']
TARGET_COLUMNS = ['cohesion', 'syntax','vocabulary', 'phraseology', 'grammar','conventions']

In [11]:
X, Y = train[FEATURE_COLUMNS], train[TARGET_COLUMNS]


In [12]:
kfold = KFold(n_splits=5)

In [37]:
predictions = []

for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = MultiOutputRegressor(LinearRegression(), n_jobs = -1)
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
    
    

Fold #0: 0.5541318246499967
Fold #1: 0.5537700014592977
Fold #2: 0.5470261004584632
Fold #3: 0.544407325985179
Fold #4: 0.5520343641116121


In [41]:
preds_mean = np.mean(predictions, axis = 0)

In [44]:
submission = pd.DataFrame()

In [46]:
submission.loc[:,'text_id'] = test.text_id

In [49]:
submission.loc[:, TARGET_COLUMNS] = preds_mean

In [62]:
submission.loc[:, TARGET_COLUMNS] = (submission.loc[:, TARGET_COLUMNS] * 2).round() / 2

In [63]:
submission

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,3.0,3.0,3.0,3.0,2.5,3.0
1,000BAD50D026,2.5,2.5,2.5,2.5,2.5,2.5
2,00367BB2546B,3.5,3.5,3.5,3.5,3.5,3.5


In [64]:
submission.to_csv('submission.csv',index=False)

In [ ]:
# Score .54 LOL